In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from anndata import read_h5ad
from thalamus_merfish_analysis import abc_load
from thalamus_merfish_analysis import ccf_registration as ccf



In [3]:
df_full = abc_load.get_combined_metadata(version="20230830")


In [5]:
df_neuronal = df_full.loc[lambda df: df['class'].str.contains('Glut') | 
                df['class'].str.contains('GABA')]
adata_ref = read_h5ad(abc_load._THALAMUS_ANNDATA_PATH, backed='r')
ref_index = adata_ref.obs_names.intersection(df_neuronal.index)

In [9]:
th_frac = df_neuronal.groupby('subclass', observed=True).apply(lambda x: len(x.index.intersection(ref_index))/len(x))

th_frac2 = df_neuronal.groupby('subclass', observed=True).apply(lambda x: ((x['parcellation_division'] == 'TH') |
                                                 (x['parcellation_structure'] == 'ZI')).sum() / len(x))

In [10]:
th_frac.sort_values(ascending=False).head(20)

In [11]:
th_subclasses = th_frac.loc[lambda x: x>0.33].index
th_subclasses2 = th_frac2.loc[lambda x: x>0.33].index
th_subclasses.equals(th_subclasses2)

In [12]:
th_subclasses

## Validation

In [4]:
coords_ccf = ['x_ccf', 'y_ccf', 'z_ccf']
coords_rec = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
coords_sec = ['x_section', 'y_section', 'z_section']


In [ ]:

common_index = df_full.index.intersection(df_new.index)
(df_full.loc[common_index, coords_ccf+coords_from+img_coords] == 
 df_new.loc[common_index, coords_ccf+coords_from+img_coords]).all()


In [27]:
(df_full.loc[common_index, img_coords] != 
 df_new.loc[common_index, img_coords]).any(axis=1).mean()

In [31]:
((df_full.loc[common_index, img_coords] - 
 df_new.loc[common_index, img_coords]).round(3) > 0).any(axis=1).mean()

In [38]:
t1 = ccf.calculate_affine_transform(df_full, coords_from, coords_ccf)
t2 = ccf.calculate_affine_transform(df_full, coords_from, coords_ccf)
np.abs(t1-t2)

In [86]:
from importlib import reload
reload(ccf)

In [49]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import scipy.linalg as linalg


In [79]:

df = df_full
coords_to = coords_ccf
coords_from = coords_rec
samples = np.random.randint(0, len(df), 200)
to = ccf.to_hom(df[coords_to].iloc[samples].values)
x, res, rank, s = linalg.lstsq(ccf.to_hom(df[coords_from].iloc[samples].values), 
             to)
    

In [80]:
res / (len(to)*np.var(to, axis=0))

## CCF images

trying to get higher resolution for quicknii

### old atlas nrrd to nifti

In [8]:
import nibabel as nib
import nrrd

In [9]:

# from http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/

# Get a list of .nrrd files in a directory
nrrd_file = '../data/annotation_10.nrrd'
# Read the .nrrd file
data, header = nrrd.read(nrrd_file)


In [10]:
data.dtype

In [12]:
data2 = np.moveaxis(np.flip(np.flip(data, 0), 1), range(3), (1, 2, 0))

In [17]:
data2 = np.array(data2, dtype=np.dtype('<u4'))
data2.dtype

In [14]:
np.array([456., 528., 320.])*2.5

In [47]:
np.array(data2.shape)/2.5

In [ ]:
nifti_img.g

In [39]:
nifti_img = nib.Nifti1Image(data2, affine=np.eye(4))


In [42]:
# Save the NIfTI1Image object as .nii file
nib.save(nifti_img, '../data/labels.nii.gz')

In [19]:
im_path = "/data/labels.nii.gz"
img = nib.load(im_path)

In [44]:
print(nifti_img.header)

In [ ]:
print(img.header)

In [27]:
img.shape

In [36]:
img.affine

In [28]:
img.header.get_zooms()

In [22]:
img.get_data_dtype()

### Nibabel nii load from atlas

In [11]:
im_path = "/data/abc_atlas/image_volumes/Allen-CCF-2020/20230630/annotation_10.nii.gz"
img = nib.load(im_path)

In [13]:
img.shape

In [14]:
img.affine

In [15]:
img.header.get_xyzt_units()

### save new IDs / colors csv

In [1]:
import pandas as pd

In [7]:
parcellation = pd.read_csv("../data/allen-brain-cell-atlas/metadata/Allen-CCF-2020/20230630/parcellation_to_parcellation_term_membership.csv")

In [ ]:
cols = ['red','green','blue', 'parcellation_term_name']
old_atlas = lambda df: df.loc[df['parcellation_term_label'].str.contains('2017')]
pick_finest = lambda df: df.loc[df['term_set_order'] == df['term_set_order'].max()]
parcellation_finest = parcellation.groupby('parcellation_index').apply(pick_finest)

In [ ]:
parcellation_finest = parcellation.groupby('parcellation_index').apply(pick_finest)


In [26]:
parcellation.head()

## making subclass images

In [45]:

def preproc_data(df, nn_classes, img_coords, ref_spatial, ref_subset=None):
    # df = df_full.loc[lambda df: df[img_coords[0]].notna()]
    if ref_subset is None:
        ref_subset = ref_spatial
    df = ccf.subset_to_ref_bounds(df, img_coords, ref_spatial)
    ref_subset = df.index.intersection(ref_subset)
    
    slice_label = 'slice_int'
    df[slice_label] = df['z_section'].apply(lambda x: str(int(x*10)))

    working_label = "thal_class"
    img_label = 'subclass_int'
    df[working_label] = df["class"]
    df.loc[ref_subset, working_label] = df.loc[ref_subset, "subclass"]

    all_labels = df[working_label].unique()
    focus_labels = df.loc[ref_subset, working_label].value_counts().index
    other_labels = df.loc[df.index.difference(ref_subset)].loc[
        lambda df: ~df['class'].isin(nn_classes), working_label].value_counts().index

    label_order = [""] + list(focus_labels) + nn_classes + list(other_labels)
    label_map = {x: label_order.index(x) for x in all_labels}
    df[img_label] = df[working_label].map(label_map)
    cmap = ccf.cmap_with_emphasis(len(focus_labels), n_secondary=len(nn_classes))
    return df, cmap

In [30]:
slice_label = 'slice_int'

working_label = "thal_class"
img_label = 'subclass_int'

img_coords = ['x_section', 'y_section', 'z_section']
coords_from = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']

In [112]:
nn_classes = ["31 OPC-Oligo",
              "30 Astro-Epen",
              "33 Vascular",
              "34 Immune"]
df, cmap = preproc_data(df_full, nn_classes, img_coords, ref_index)
out = ccf.export_to_quicknii(df, 'brain3_20220830', cmap=cmap,
                       img_coords=img_coords, coords_from=coords_from,
                       img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_affine_thal_focus_20220630', 
                   save_json=True, save_images=False)

In [39]:
nn_classes = ["31 OPC-Oligo",
              "30 Astro-Epen",
              "33 Vascular",
              "34 Immune"]
ref_subclasses = df_neuronal.loc[lambda df: df['subclass'].isin(th_subclasses)].index
df, cmap = preproc_data(df_full, nn_classes, img_coords, ref_index, ref_subclasses)


In [ ]:

out = ccf.export_to_quicknii(df, 'brain3_20220830', cmap=cmap,
                       img_coords=img_coords, coords_from=coords_from,
                       img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_affine_thal_subclasses_20220630', 
                   save_json=True, save_images=True)

In [113]:
# old
nn_classes = [
    "28 Astro-Epen",
    "29 Oligo",
    "31 Vascular",
    "32 Immune"
]
df, cmap = preproc_data(df_full, nn_classes, img_coords, ref_index)
out = ccf.export_to_quicknii(df, 'brain3_20220630_2', cmap=cmap,
                       img_coords=img_coords, coords_from=coords_from,
                       img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_affine_thal_focus_20220630', 
                   save_json=True, save_images=False)

In [41]:
coords_from = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
img_coords = ['x_section', 'y_section', 'z_section']
ccf.export_to_quicknii(df, 'brain3_20220630', cmap=cmap,
                       img_coords=img_coords, coords_from=coords_from,
                       img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_affine_thal_focus_20220630', save_images=True);

In [ ]:
coords_from = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
img_coords = ['x_section', 'y_section', 'z_section']
ccf.export_to_quicknii(df, 'brain3', cmap=cmap,
                       img_coords=img_coords, coords_from=coords_from,
                       img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_affine_thal_focus', save_images=True);

In [10]:
nn_classes = ["31 OPC-Oligo",
              "30 Astro-Epen",
              "33 Vascular",
              "34 Immune"]
df, cmap = preproc_data(df_full, nn_classes, img_coords, ref_index)
img_coords = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
ccf.export_to_quicknii(df, 'brain3_subclass', cmap=cmap, img_coords=img_coords, img_label=img_label, slice_label=slice_label, scale=25,
                   path='../results/qn_25_reconstructed', save_images=True);

In [8]:
img_coords = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
df[img_coords].agg(['min','max'])

In [ ]:
df[img_coords].agg(['min','max'])

In [9]:
df[['x_ccf', 'y_ccf', 'z_ccf']].agg(['min','max'])

In [ ]:
df[['x_ccf', 'y_ccf', 'z_ccf']].agg(['min','max'])

In [14]:
img_coords = ['x_reconstructed', 'y_reconstructed', 'z_reconstructed']
dfslice = df.query("brain_section_label=='C57BL6J-638850.43'").copy()

In [39]:
dfimg, coords_scaled, coords_int = ccf.add_rescaled_coords(dfslice, img_coords[:2], xy_res=0.0025,
                                                    ref_subset=ref_index.intersection(dfslice.index))


In [37]:
nx, ny = dfimg[coords_int].max() + 1
(nx, ny)

In [40]:
img = ccf.rasterize_by_dilation(dfimg, coords_int, label=img_label, shape=(ny, nx))

plt.imshow(img, cmap=ccf.CMAP, interpolation='nearest')
# plt.imsave("test.tiff", img.T, cmap=cmap)
plt.show()

In [21]:
df = adata.obs
# add_rescaled_coords(df, xy_res=0.0025)
x, y, z = ('x', 'y', 'z_reconstructed')
to_ccf = calculate_affine_transform(df, (x,y,z))

In [26]:
z = 7.8
origin = np.array([[0, 1, z]])
origin_ccf = np.matmul(to_hom(origin), to_ccf)
origin_ccf

In [35]:
origin_qn = np.matmul(origin_ccf, ccf_to_qn)
origin_qn

In [83]:
np.linalg.inv(np.array([[1, 0, 1], [0, 1, 1], [0, 0, 1]]))